In [1]:
# Setup
import sys
from pathlib import Path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from core.multi_asset_loader import load_assets
from core.multi_asset_signal import SingleAssetWrapper
from signals.momentum import MomentumSignalV2
from core.portfolio_manager import PortfolioManager, PortfolioConfig

print("✅ Imports successful")

✅ Imports successful


---

## Step 1: Load Multi-Asset Data

We'll trade 3 assets: ES (S&P), GC (Gold), NQ (Nasdaq)

In [2]:
# Load data for 3 assets
TICKERS = ['ES', 'GC', 'NQ']
prices = load_assets(
    tickers=TICKERS,
    start_date='2024-01-01',
    end_date='2024-12-31'
)

print("\n📊 Loaded Data:")
for ticker, df in prices.items():
    print(f"  {ticker}: {len(df)} days, {df.index[0].date()} to {df.index[-1].date()}")
    print(f"    Current price: ${df['Close'].iloc[-1]:,.2f}")


Loading 3 assets: ['ES', 'GC', 'NQ']
  📡 CSV data is 330 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-26)
✓ ES: 6362 rows, 2000-09-18 to 2025-11-26
  📡 CSV data is 330 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-26)
✓ GC: 6334 rows, 2000-08-30 to 2025-11-26
  📡 CSV data is 330 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-26)
✓ ES: 6362 rows, 2000-09-18 to 2025-11-26
  📡 CSV data is 330 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-26)
✓ GC: 6334 rows, 2000-08-30 to 2025-11-26
  📡 CSV data is 330 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-26)
✓ NQ: 6362 rows, 2000-09-18 to 2025-11-26

Common date range: 2000-09-18 to 2025-11-26
Filtered date range: 2024-01-01 to 2024-12-31

Total unique dates: 252

ALIGN

---

## Step 2: Generate Signals for Each Asset

**Source:** `core/multi_asset_signal.py` → `SingleAssetWrapper`

This applies the same momentum strategy to each asset independently.

In [3]:
# Generate signals
signal_gen = MomentumSignalV2(lookback=120, sma_filter=200)
multi_signal = SingleAssetWrapper(signal_gen)
signals = multi_signal.generate(prices)

print("\n🎯 Signal Statistics:")
for ticker in TICKERS:
    sig = signals[ticker]['Signal']
    n_long = (sig == 1).sum()
    print(f"  {ticker}: {n_long}/{len(sig)} days long ({n_long/len(sig)*100:.1f}%)")

# Show current signals
print("\n📍 Current Signals:")
for ticker in TICKERS:
    current_signal = signals[ticker]['Signal'].iloc[-1]
    signal_text = 'LONG' if current_signal == 1 else 'FLAT'
    print(f"  {ticker}: {signal_text}")


🎯 Signal Statistics:
  ES: 32/252 days long (12.7%)
  GC: 32/252 days long (12.7%)
  NQ: 32/252 days long (12.7%)

📍 Current Signals:
  ES: LONG
  GC: LONG
  NQ: LONG


---

## Step 3: Create PortfolioManager

**Source:** `core/portfolio_manager.py` lines 17-28

The config controls:
- `initial_capital` - Starting cash
- `rebalance_threshold` - When to rebalance (5% drift)
- `transaction_cost_bps` - Cost per trade (3 bps)
- `risk_manager` - Optional risk overlay (we'll skip for now)

In [4]:
# Create configuration
config = PortfolioConfig(
    initial_capital=100000,
    rebalance_threshold=0.05,  # 5% drift triggers rebalance
    transaction_cost_bps=3.0,
    risk_manager=None  # No risk management yet
)

# Create portfolio manager
pm = PortfolioManager(config)

print("\n💼 PortfolioManager Created:")
print(f"  Initial Capital: ${pm.config.initial_capital:,.2f}")
print(f"  Cash: ${pm.cash:,.2f}")
print(f"  Portfolio Value: ${pm.portfolio_value:,.2f}")
print(f"  Positions: {len(pm.positions)}")


💼 PortfolioManager Created:
  Initial Capital: $100,000.00
  Cash: $100,000.00
  Portfolio Value: $100,000.00
  Positions: 0


---

## Step 4: Initialize Positions

**Source:** `core/portfolio_manager.py` lines 57-96 → `initialize_positions()`

**What Happens:**
1. Count active signals (where signal != 0)
2. Calculate equal weight per asset: 1 / n_active
3. Allocate capital: weight × initial_capital
4. Buy shares: allocation / price
5. Reduce cash by allocation

In [5]:
# Get initial prices and signals
initial_prices = {ticker: prices[ticker]['Close'].iloc[0] for ticker in TICKERS}
initial_signals = {ticker: signals[ticker]['Signal'].iloc[0] for ticker in TICKERS}

print("\n📌 Initial State (Day 1):")
print(f"{'Ticker':<8} {'Price':>10} {'Signal':>8}")
print("-" * 30)
for ticker in TICKERS:
    print(f"{ticker:<8} ${initial_prices[ticker]:>9,.2f} {initial_signals[ticker]:>8}")

# Initialize positions
pm.initialize_positions(initial_prices, initial_signals)

print("\n✅ Positions Initialized:")
print(f"\n{'Ticker':<8} {'Shares':>12} {'Value':>15} {'Weight':>10}")
print("-" * 50)
for ticker in TICKERS:
    pos = pm.positions[ticker]
    print(f"{ticker:<8} {pos['shares']:>12,.4f} ${pos['value']:>14,.2f} {pos['weight']:>9.1%}")

print(f"\n💰 Cash Remaining: ${pm.cash:,.2f}")
print(f"📊 Portfolio Value: ${pm.portfolio_value:,.2f}")


📌 Initial State (Day 1):
Ticker        Price   Signal
------------------------------
ES       $ 4,787.25      0.0
GC       $ 2,064.40      0.0
NQ       $16,720.00      0.0

✅ Positions Initialized:

Ticker         Shares           Value     Weight
--------------------------------------------------
ES             0.0000 $          0.00      0.0%
GC             0.0000 $          0.00      0.0%
NQ             0.0000 $          0.00      0.0%

💰 Cash Remaining: $100,000.00
📊 Portfolio Value: $100,000.00


---

## Step 5: Update Position Values

**Source:** `core/portfolio_manager.py` lines 97-118 → `update_positions()`

As prices change, position values change but shares stay constant.

In [6]:
# Get prices 10 days later
day_10_prices = {ticker: prices[ticker]['Close'].iloc[10] for ticker in TICKERS}

print("\n📈 Price Changes (Day 1 → Day 10):")
print(f"{'Ticker':<8} {'Day 1':>10} {'Day 10':>10} {'Change':>10}")
print("-" * 42)
for ticker in TICKERS:
    p1 = initial_prices[ticker]
    p2 = day_10_prices[ticker]
    change = (p2/p1 - 1) * 100
    print(f"{ticker:<8} ${p1:>9,.2f} ${p2:>9,.2f} {change:>9.2f}%")

# Update positions with new prices
pm.update_positions(day_10_prices)

print("\n✅ Updated Position Values:")
print(f"\n{'Ticker':<8} {'Shares':>12} {'Old Value':>15} {'New Value':>15} {'P&L':>12}")
print("-" * 70)

for ticker in TICKERS:
    pos = pm.positions[ticker]
    old_value = pos['shares'] * initial_prices[ticker]
    new_value = pos['value']
    pnl = new_value - old_value
    print(f"{ticker:<8} {pos['shares']:>12,.4f} ${old_value:>14,.2f} ${new_value:>14,.2f} ${pnl:>11,.2f}")

print(f"\n💰 Cash (unchanged): ${pm.cash:,.2f}")
print(f"📊 Portfolio Value: ${pm.portfolio_value:,.2f}")
print(f"💵 Total P&L: ${pm.portfolio_value - config.initial_capital:,.2f}")


📈 Price Changes (Day 1 → Day 10):
Ticker        Day 1     Day 10     Change
------------------------------------------
ES       $ 4,787.25 $ 4,771.25     -0.33%
GC       $ 2,064.40 $ 2,002.60     -2.99%
NQ       $16,720.00 $16,869.75      0.90%

✅ Updated Position Values:

Ticker         Shares       Old Value       New Value          P&L
----------------------------------------------------------------------
ES             0.0000 $          0.00 $          0.00 $       0.00
GC             0.0000 $          0.00 $          0.00 $       0.00
NQ             0.0000 $          0.00 $          0.00 $       0.00

💰 Cash (unchanged): $100,000.00
📊 Portfolio Value: $100,000.00
💵 Total P&L: $0.00


---

## Step 6: Handle Signal Changes

**Source:** `core/portfolio_manager.py` lines 310-449 → `update_signals()`

**Logic:**
1. Detect signal changes (old signal vs new signal)
2. **Exits first** (signal 1→0 or -1→0): Sell to raise cash
3. **Entries second** (signal 0→1 or 0→-1): Buy with available cash
4. **Flips** (signal 1→-1 or -1→1): Close and reverse

**Why exits before entries?** Need cash to enter new positions!

In [8]:
# Simulate a signal change for GC: LONG (1) → FLAT (0)
date = prices['GC'].index[10]
new_prices = day_10_prices.copy()
new_signals = initial_signals.copy()
new_signals['GC'] = 1  # Exit GC position

print("\n🔄 Signal Change Detected:")
print(f"  GC: LONG (1) → FLAT (0)")
print(f"  Action: SELL all GC shares")

print(f"\n📊 Before Signal Update:")
print(f"  GC Shares: {pm.positions['GC']['shares']:.4f}")
print(f"  GC Value: ${pm.positions['GC']['value']:,.2f}")
print(f"  Cash: ${pm.cash:,.2f}")

# Update signals (this triggers trades)
pm.update_signals(new_signals, new_prices, date)

print(f"\n✅ After Signal Update:")
print(f"  GC Shares: {pm.positions['GC']['shares']:.4f}")
print(f"  GC Value: ${pm.positions['GC']['value']:,.2f}")
print(f"  Cash: ${pm.cash:,.2f}")

# Show the trade that was executed
if len(pm.trades) > 0:
    last_trade = pm.trades[-1]
    print(f"\n📝 Trade Executed:")
    print(f"  Date: {last_trade['Date'].date()}")
    print(f"  Ticker: {last_trade['Ticker']}")
    print(f"  Type: {last_trade['Type']}")
    print(f"  Shares: {last_trade['Shares']:.4f}")
    print(f"  Price: ${last_trade['Price']:,.2f}")
    print(f"  Value: ${last_trade['Value']:,.2f}")
    print(f"  Transaction Cost: ${last_trade['TransactionCost']:,.2f}")


🔄 Signal Change Detected:
  GC: LONG (1) → FLAT (0)
  Action: SELL all GC shares

📊 Before Signal Update:
  GC Shares: 0.0000
  GC Value: $0.00
  Cash: $100,000.00

✅ After Signal Update:
  GC Shares: 49.4357
  GC Value: $99,000.00
  Cash: $970.30

📝 Trade Executed:
  Date: 2024-01-17
  Ticker: GC
  Type: Entry
  Shares: 49.4357
  Price: $2,002.60
  Value: $99,000.00
  Transaction Cost: $29.70


---

## Step 7: Rebalancing Logic

**Source:** `core/portfolio_manager.py` lines 162-221 → `rebalance()`

**When:** Weights drift > threshold (e.g., 5%)

**How:**
1. Calculate total value in active positions
2. Target = equal split among active positions
3. Buy/sell to reach target weights

**Example:** If ES is 40% and GC is 20%, rebalance to 30%/30%

In [9]:
# Check if rebalance is needed
print("\n⚖️ Checking Rebalance Conditions:")
print(f"  Threshold: {config.rebalance_threshold:.1%}")

# Show current weights
active_value = sum(pos['value'] for ticker, pos in pm.positions.items() if pos['shares'] != 0)
print(f"\n📊 Current Weights:")
for ticker in TICKERS:
    pos = pm.positions[ticker]
    if pos['shares'] != 0:
        weight = pos['value'] / active_value if active_value > 0 else 0
        print(f"  {ticker}: {weight:.1%}")

# Check rebalance
needs_rebalance = pm.check_rebalance_needed(new_signals)
print(f"\n{'✅' if needs_rebalance else '❌'} Rebalance Needed: {needs_rebalance}")

if needs_rebalance:
    print("\n🔄 Executing Rebalance...")
    pm.rebalance(new_prices, new_signals, date)
    
    print(f"\n✅ After Rebalance:")
    active_value = sum(pos['value'] for ticker, pos in pm.positions.items() if pos['shares'] != 0)
    for ticker in TICKERS:
        pos = pm.positions[ticker]
        if pos['shares'] != 0:
            weight = pos['value'] / active_value if active_value > 0 else 0
            print(f"  {ticker}: {weight:.1%}")


⚖️ Checking Rebalance Conditions:
  Threshold: 5.0%

📊 Current Weights:
  GC: 100.0%

❌ Rebalance Needed: False


---

## Step 8: Equity Curve Tracking

**Source:** `core/portfolio_manager.py` line 128 → `get_portfolio_state()`

Every day, we record:
- Date
- Total portfolio value
- Cash
- Positions (snapshot)

In [10]:
# Show equity curve structure
if len(pm.equity_curve) > 0:
    print("\n📈 Equity Curve Sample:")
    print(f"\nTotal points: {len(pm.equity_curve)}")
    print(f"\n{pm.equity_curve[0]}")
else:
    print("\n⚠️ Equity curve not populated yet (this is manual mode)")
    print("   In run_multi_asset_backtest(), equity curve is built automatically")


⚠️ Equity curve not populated yet (this is manual mode)
   In run_multi_asset_backtest(), equity curve is built automatically


---

## 🎓 Key Takeaways

### PortfolioManager Architecture:

```
┌─────────────────────────────────────┐
│      PortfolioManager               │
│                                     │
│  State:                             │
│    • cash                           │
│    • positions = {ticker: {...}}   │
│    • trades = [...]                │
│    • equity_curve = [...]          │
│                                     │
│  Methods:                           │
│    • initialize_positions()        │
│    • update_positions()            │
│    • update_signals()              │
│    • rebalance()                   │
│    • calculate_metrics()           │
└─────────────────────────────────────┘
```

### Trade Execution Order:
1. **Exits** - Close positions first (raises cash)
2. **Entries** - Open new positions (uses cash)
3. **Rebalances** - Adjust among active positions

### Position Tracking:
```python
positions = {
    'ES': {
        'shares': 2.5,
        'entry_price': 5500.0,
        'current_price': 5650.0,
        'value': 14125.0,  # shares × current_price
        'weight': 0.33
    }
}
```

### Critical Insight:
⚠️ **Positions update automatically with prices**, but **cash only changes on trades**!

---

## ✅ Next Step

**Open: `03_risk_manager_explained.ipynb`**

Learn how RiskManager adds position sizing, drawdown stops, and correlation limits!